In [1]:
!pip install catboost

     |████████████████████████████████| 66.1MB 55kB/s 


In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 2.1MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=9d098b51a3cd34e89fcbedef6c21d5fe69105fe2f1c1279383e38149556aa118
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [3]:
!mkdir /content/.kaggle

In [4]:
!mkdir /root/.kaggle

In [5]:
import json
token = {"username":"sigmafactor","key":"53ecd9782335ba40d8820c882d02cfe9"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [6]:
!cp /content/.kaggle/kaggle.json  ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d nehaprabhavalkar/av-healthcare-analytics-ii

  0% 0.00/6.54M [00:00<?, ?B/s]
100% 6.54M/6.54M [00:00<00:00, 72.2MB/s]


In [8]:
!unzip /content/av-healthcare-analytics-ii.zip

Archive:  /content/av-healthcare-analytics-ii.zip
  inflating: healthcare/sample_sub.csv  
  inflating: healthcare/test_data.csv  
  inflating: healthcare/train_data.csv  
  inflating: healthcare/train_data_dictionary.csv  


In [9]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [61]:
feat_info = pd.read_csv('/content/healthcare/train_data_dictionary.csv')
data_train = pd.read_csv('/content/healthcare/train_data.csv')
data_test = pd.read_csv('/content/healthcare/test_data.csv')

In [62]:
feat_info

,Column,Description
0,case_id,Case_ID registered in Hospital
1,Hospital_code,Unique code for the Hospital
2,Hospital_type_code,Unique code for the type of Hospital
3,City_Code_Hospital,City Code of the Hospital
4,Hospital_region_code,Region Code of the Hospital
5,Available Extra Rooms in Hospital,Number of Extra rooms available in the Hospital
6,Department,Department overlooking the case
7,Ward_Type,Code for the Ward type
8,Ward_Facility_Code,Code for the Ward Facility
9,Bed Grade,Condition of Bed in the Ward


In [63]:
label_encoder = LabelEncoder()
Y_train = data_train.iloc[:,-1]
Y_train = label_encoder.fit_transform(Y_train)

In [64]:
num_var= [feat for feat in data_train.columns if (data_train[feat].dtype == int) or (data_train[feat].dtype == float)]
obj_var = list(set(data_train.columns)- set(num_var))
obj_var.remove('Stay')

In [65]:
obj_var.remove('Severity of Illness')
obj_var.remove('Type of Admission')
obj_var.remove('Age')

In [66]:
data_combined = pd.concat([data_train.iloc[:, :-1],data_test], ignore_index=True)

In [67]:
data_combined['Age'] = label_encoder.transform(data_combined['Age'])

In [68]:
for feat in obj_var:
  temp = pd.get_dummies(data_combined[feat], prefix=feat, dtype= int)
  data_combined = pd.concat([data_combined,temp], axis= 1)
  data_combined = data_combined.drop(feat, axis=1)

In [69]:
data_combined['Severity of Illness'].replace(['Extreme', 'Moderate', 'Minor'], [3,2,1], inplace=True)
data_combined['Type of Admission'].replace(['Emergency', 'Trauma','Urgent'], [3,2,1], inplace=True)
data_combined

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Hospital_type_code_a,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Ward_Type_P,Ward_Type_Q,Ward_Type_R,Ward_Type_S,Ward_Type_T,Ward_Type_U,Hospital_region_code_X,Hospital_region_code_Y,Hospital_region_code_Z
0,1,8,3,3,2.0,31397,7.0,3,3,2,5,4911.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
1,2,2,5,2,2.0,31397,7.0,2,3,2,5,5954.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
2,3,10,1,2,2.0,31397,7.0,2,3,2,5,4745.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
3,4,26,2,2,2.0,31397,7.0,2,3,2,5,7272.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
4,5,26,2,2,2.0,31397,7.0,2,3,2,5,5558.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,455491,11,2,4,3.0,41160,3.0,3,1,4,4,6313.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
455491,455492,25,1,2,4.0,30985,7.0,3,2,2,0,3510.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
455492,455493,30,3,2,4.0,81811,12.0,1,1,2,0,7190.0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
455493,455494,5,1,2,4.0,57021,10.0,2,1,2,4,5435.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0


In [70]:
data_combined['Bed Grade'].replace(np.nan , 2, inplace=True)

In [71]:
data_combined['Charges_per_Bed_grade'] = data_combined['Admission_Deposit']/data_combined['Bed Grade']
data_combined['Severity_by_visit'] = data_combined['Visitors with Patient']/data_combined['Severity of Illness']
data_combined['Admission_by_visit'] = data_combined['Visitors with Patient']/data_combined['Type of Admission']
data_combined['No_of_visits'] = data_combined.groupby('patientid')['patientid'].transform('size')

In [72]:
data_combined.corr()

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Hospital_type_code_a,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Ward_Type_P,Ward_Type_Q,Ward_Type_R,Ward_Type_S,Ward_Type_T,Ward_Type_U,Hospital_region_code_X,Hospital_region_code_Y,Hospital_region_code_Z,Charges_per_Bed_grade,Severity_by_visit,Admission_by_visit,No_of_visits
case_id,1.000000,-0.024139,-0.008247,0.023878,0.011787,-0.000235,0.038053,-0.044755,-0.087231,0.000818,-0.066957,-0.029820,0.003433,-0.012613,0.013884,-0.007702,-0.007315,-0.002170,-0.005512,0.015082,0.002860,-0.010180,-0.006337,0.010568,0.019407,0.004400,-0.025119,0.001041,0.000450,0.000900,0.001541,0.016664,-0.005207,-0.012195,-0.003573,-0.003136,0.002966,-0.012565,0.011672,-0.019076,0.053529,0.041088,-0.214408
Hospital_code,-0.024139,1.000000,0.127760,-0.058744,-0.014664,0.002912,-0.015260,0.014603,0.029914,-0.028168,0.017573,0.046347,0.008199,0.001598,-0.012271,0.010686,0.002168,-0.072857,0.261370,-0.143568,-0.295059,-0.027901,0.294774,0.049183,0.016815,-0.002147,0.158890,0.128045,-0.114475,-0.120931,0.050657,-0.207152,0.090198,0.116953,-0.044694,-0.006139,0.006107,0.198316,-0.249812,0.031440,-0.040550,-0.030789,0.010151
City_Code_Hospital,-0.008247,0.127760,1.000000,-0.045313,-0.048507,0.001617,-0.025153,0.016754,0.025952,0.018377,0.022641,-0.034767,0.000405,-0.046642,0.025728,0.009565,0.002223,0.122226,-0.090188,-0.152101,0.248680,-0.352693,0.253910,0.157687,-0.177426,0.543805,0.253442,-0.393906,-0.556557,0.328436,-0.049155,-0.133316,0.111631,0.012635,0.129806,-0.002925,-0.110179,0.213965,-0.124798,0.017962,-0.002906,0.011115,0.023391
Available Extra Rooms in Hospital,0.023878,-0.058744,-0.045313,1.000000,-0.115326,0.001551,-0.008715,0.015661,-0.004505,0.095790,-0.131132,-0.143479,-0.025344,-0.103345,0.114432,-0.043258,-0.007094,0.052036,0.026630,-0.008063,-0.000589,-0.047623,-0.088893,-0.044991,-0.016765,-0.079302,0.007338,0.048375,-0.003082,0.022091,0.225444,0.323036,-0.086762,-0.316156,-0.031093,0.000237,0.009741,-0.009545,-0.000414,0.028141,0.085646,0.079684,-0.055628
Bed Grade,0.011787,-0.014664,-0.048507,-0.115326,1.000000,0.000580,-0.006967,-0.126118,-0.283624,0.089360,0.000056,0.071756,0.023948,0.142591,-0.101589,-0.007982,-0.022446,-0.040714,0.027585,0.012402,0.003520,0.027213,0.008670,-0.045561,0.007886,-0.021722,-0.145794,0.009009,0.048023,0.060058,0.013415,-0.004270,0.019301,-0.020125,-0.006822,-0.002226,0.082058,-0.101091,0.021777,-0.780919,0.231062,0.132786,0.006929
patientid,-0.000235,0.002912,0.001617,0.001551,0.000580,1.000000,0.000979,-0.002314,-0.006589,0.002865,-0.000524,0.000033,0.003265,-0.000211,-0.003585,0.003450,-0.000128,0.002085,0.000954,-0.000838,-0.000867,-0.002172,-0.000546,-0.002133,-0.001250,-0.002420,0.004430,-0.000386,-0.001179,0.000654,0.001104,-0.002993,0.003183,-0.000736,0.000555,-0.001607,0.000739,0.001005,-0.002143,0.000616,0.005891,0.003698,-0.002519
City_Code_Patient,0.038053,-0.015260,-0.025153,-0.008715,-0.006967,0.000979,1.000000,0.027456,-0.020288,-0.012611,-0.018199,0.025665,-0.007522,-0.012562,0.002888,0.012746,0.001518,-0.051952,-0.025570,0.065304,0.048758,-0.019332,0.026536,0.015897,0.045435,0.073267,0.055935,0.045877,-0.022797,-0.129167,0.015092,0.017745,-0.066439,0.047799,0.026021,0.001537,-0.172199,0.118154,0.068843,0.017841,0.003573,-0.018043,-0.061312
Type of Admission,-0.044755,0.014603,0.016754,0.015661,-0.126118,-0.002314,0.027456,1.000000,-0.024140,0.097634,0.063642,-0.042687,-0.014909,-0.082553,0.034613,0.029067,0.065267,0.112659,-0.051044,-0.055955,-0.043078,0.005084,-0.009398,-

In [73]:
data_combined['City_Code_Patient'].replace(np.nan, 8, inplace=True)

In [74]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2, interaction_only=True)
temp = pd.DataFrame(data=poly.fit_transform(data_combined.iloc[:, [1,2]]))


In [75]:
data_combined = pd.concat([data_combined, temp], axis=1)
data_combined

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Hospital_type_code_a,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Ward_Type_P,Ward_Type_Q,Ward_Type_R,Ward_Type_S,Ward_Type_T,Ward_Type_U,Hospital_region_code_X,Hospital_region_code_Y,Hospital_region_code_Z,Charges_per_Bed_grade,Severity_by_visit,Admission_by_visit,No_of_visits,0,1,2,3
0,1,8,3,3,2.0,31397,7.0,3,3,2,5,4911.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,2455.500000,0.666667,0.666667,14,1.0,8.0,3.0,24.0
1,2,2,5,2,2.0,31397,7.0,2,3,2,5,5954.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,2977.000000,0.666667,1.000000,14,1.0,2.0,5.0,10.0
2,3,10,1,2,2.0,31397,7.0,2,3,2,5,4745.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,2372.500000,0.666667,1.000000,14,1.0,10.0,1.0,10.0
3,4,26,2,2,2.0,31397,7.0,2,3,2,5,7272.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,3636.000000,0.666667,1.000000,14,1.0,26.0,2.0,52.0
4,5,26,2,2,2.0,31397,7.0,2,3,2,5,5558.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2779.000000,0.666667,1.000000,14,1.0,26.0,2.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,455491,11,2,4,3.0,41160,3.0,3,1,4,4,6313.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,2104.333333,4.000000,1.333333,1,1.0,11.0,2.0,22.0
455491,455492,25,1,2,4.0,30985,7.0,3,2,2,0,3510.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,877.500000,1.000000,0.666667,1,1.0,25.0,1.0,25.0
455492,455493,30,3,2,4.0,81811,12.0,1,1,2,0,7190.0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1797.500000,2.000000,2.000000,1,1.0,30.0,3.0,90.0
455493,455494,5,1,2,4.0,57021,10.0,2,1,2,4,5435.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1358.750000,2.000000,1.000000,1,1.0,5.0,1.0,5.0


In [76]:
data_combined.drop(['patientid', 'case_id'], axis=1, inplace=True)
X_train_init = data_combined.iloc[:318438, :]
X_test_init = data_combined.iloc[318438:, :]

In [77]:
def reduce_mem_usage_colwise(col):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    col_type = col.dtypes
    if col_type in numerics:
        c_min = col.min()
        c_max = col.max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                col = col.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                col = col.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                col = col.astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                col = col.astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                col = col.astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                col = col.astype(np.float32)
            else:
                col = col.astype(np.float64)    
    return col
#my_list = X_train_init.columns
#X_train_init[my_list] = X_train_init[my_list].apply(lambda x: reduce_mem_usage_colwise(x))
#X_test_init[my_list] = X_test_init[my_list].apply(lambda x: reduce_mem_usage_colwise(x))

In [78]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X_train_init, Y_train)
X_train_fs = fs.transform(X_train_init)
#X_test_fs = fs.transform(data_test.iloc[:, 1:])
for i in range(len(fs.scores_)):
	print('%d %s: %f' % (i,X_train_init.columns[i], fs.scores_[i]))
select_feat = [i for i in range(len(fs.scores_)) if fs.scores_[i]>1000]

0 Hospital_code: 3476.280414
1 City_Code_Hospital: 2116.634886
2 Available Extra Rooms in Hospital: 2949.504913
3 Bed Grade: 1268.505260
4 City_Code_Patient: 1185.902384
5 Type of Admission: 686.824769
6 Severity of Illness: 1439.026472
7 Visitors with Patient: 100797.616851
8 Age: 2546.358781
9 Admission_Deposit: 1116661.781151
10 Department_TB & Chest disease: 58.307534
11 Department_anesthesia: 986.292572
12 Department_gynecology: 107.693524
13 Department_radiotherapy: 360.650834
14 Department_surgery: 105.978131
15 Hospital_type_code_a: 1421.222731
16 Hospital_type_code_b: 1728.790938
17 Hospital_type_code_c: 145.057655
18 Hospital_type_code_d: 260.898311
19 Hospital_type_code_e: 109.414071
20 Hospital_type_code_f: 1278.655485
21 Hospital_type_code_g: 596.601254
22 Ward_Facility_Code_A: 183.310245
23 Ward_Facility_Code_B: 764.688057
24 Ward_Facility_Code_C: 3213.140698
25 Ward_Facility_Code_D: 2440.925976
26 Ward_Facility_Code_E: 615.159477
27 Ward_Facility_Code_F: 290.245691
28 Wa

In [79]:
#from sklearn.model_selection import train_test_split

#X_train, X_valid, y_train, y_valid = train_test_split(X_train_init.iloc[:, select_feat],Y_train)

In [37]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

scalar = StandardScaler()
X_train = pd.DataFrame(scalar.fit_transform(X_train_init.iloc[:, select_feat]))
X_test = pd.DataFrame(scalar.transform(X_test_init.iloc[:, select_feat]))

cat = CatBoostClassifier(custom_metric='Accuracy')
cat.fit(X_train, Y_train)

Learning rate set to 0.106164
0:	learn: 2.2207175	total: 565ms	remaining: 9m 24s
1:	learn: 2.1055452	total: 1.07s	remaining: 8m 54s
2:	learn: 2.0194751	total: 1.56s	remaining: 8m 38s
3:	learn: 1.9517133	total: 2.04s	remaining: 8m 28s
4:	learn: 1.8994015	total: 2.54s	remaining: 8m 25s
5:	learn: 1.8551848	total: 3.03s	remaining: 8m 22s
6:	learn: 1.8196545	total: 3.52s	remaining: 8m 19s
7:	learn: 1.7863365	total: 4.02s	remaining: 8m 18s
8:	learn: 1.7603939	total: 4.51s	remaining: 8m 16s
9:	learn: 1.7382625	total: 5.01s	remaining: 8m 16s
10:	learn: 1.7173763	total: 5.51s	remaining: 8m 15s
11:	learn: 1.7009672	total: 6s	remaining: 8m 14s
12:	learn: 1.6854222	total: 6.49s	remaining: 8m 12s
13:	learn: 1.6733195	total: 6.99s	remaining: 8m 12s
14:	learn: 1.6613025	total: 7.49s	remaining: 8m 11s
15:	learn: 1.6517993	total: 7.99s	remaining: 8m 11s
16:	learn: 1.6430926	total: 8.49s	remaining: 8m 11s
17:	learn: 1.6347424	total: 8.99s	remaining: 8m 10s
18:	learn: 1.6290745	total: 9.48s	remaining: 8m

In [38]:
cat.score(X_train, Y_train)

0.4550775975229087

In [39]:
prediction = cat.predict(X_test)
prediction = label_encoder.inverse_transform(prediction)

submission = pd.DataFrame({'case_id': data_test.case_id, 'Stay': prediction})

submission.to_csv('submission.csv',index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#SVC CLASSIFIER Taking too much time
#https://stackoverflow.com/questions/53940258/svc-classifier-taking-too-much-time-for-training
#Time for GRIDSearchSCV
#https://datascience.stackexchange.com/questions/29495/how-to-estimate-gridsearchcv-computing-time
